In [ ]:
import requests
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res = requests.get(youbike_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

from pydantic import BaseModel, Field, RootModel, field_validator

class Site(BaseModel):
    站名: str = Field(alias='sna')
    區域: str = Field(alias='sarea')
    時間: str = Field(alias='mday')
    地點: str = Field(alias='ar')
    狀態: bool = Field(alias='act')
    總共數量: int = Field(alias='total')
    可借數量: int = Field(alias='available_rent_bikes')
    可還數量: int = Field(alias='available_return_bikes')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')

    @field_validator('站名')
    @classmethod
    def get_name(cls, value: str) -> str:
        return value.split('_')[-1]

class YouBike(RootModel):
    root: list[Site]
    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, item):
        return self.root[item]
    
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()

import pandas as pd
df = pd.DataFrame(all_sites)
df

In [ ]:
#主程式要先跑過，才能在另一組執行
df[df['可借數量'] <= 3]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1423 non-null   object 
 1   區域      1423 non-null   object 
 2   時間      1423 non-null   object 
 3   地點      1423 non-null   object 
 4   狀態      1423 non-null   bool   
 5   總共數量    1423 non-null   int64  
 6   可借數量    1423 non-null   int64  
 7   可還數量    1423 non-null   int64  
 8   經度      1423 non-null   float64
 9   緯度      1423 non-null   float64
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.6+ KB


In [16]:
import numpy as np

less3_df = df.query('可借數量<=3') #query是字串
less3_df.shape

(432, 10)